In [1]:
# 라이브러리 불러오기
import urllib.request 
import json 
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns
import numpy as np

%config InlineBackend.figure_format='retina'

# 한글 쓰기
plt.rc('font', family='Malgun Gothic')
sns.set(font="Malgun Gothic",#"NanumGothicCoding", 
        rc={"axes.unicode_minus":False}, # 마이너스 부호 깨짐 현상 해결
        style='darkgrid')

In [2]:
path = '추석 연휴 시간대 예측.csv'

chu_pre = pd.read_csv(path)

chu_pre.head()

,Unnamed: 0,연도,시간대,출발영업소,도착영업소,온도,풍속,시정,교통량,통행시간(분)
0,0,2021,0,서울,부산,20.3,1.8,2000,53920,228
1,1,2021,1,서울,부산,20.1,2.0,2000,35778,240
2,2,2021,2,서울,부산,19.9,2.6,2000,26782,243
3,3,2021,3,서울,부산,19.5,2.1,2000,24182,242
4,4,2021,4,서울,부산,19.2,1.7,2000,30804,254


In [3]:
chu_pre.drop(['출발영업소','도착영업소'], axis = 1 , inplace = True)
chu_pre.head()

,Unnamed: 0,연도,시간대,온도,풍속,시정,교통량,통행시간(분)
0,0,2021,0,20.3,1.8,2000,53920,228
1,1,2021,1,20.1,2.0,2000,35778,240
2,2,2021,2,19.9,2.6,2000,26782,243
3,3,2021,3,19.5,2.1,2000,24182,242
4,4,2021,4,19.2,1.7,2000,30804,254


In [4]:
chu_pre.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46 entries, 0 to 45
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  46 non-null     int64  
 1   연도          46 non-null     int64  
 2   시간대         46 non-null     int64  
 3   온도          46 non-null     float64
 4   풍속          46 non-null     float64
 5   시정          46 non-null     int64  
 6   교통량         46 non-null     int64  
 7   통행시간(분)     46 non-null     int64  
dtypes: float64(2), int64(6)
memory usage: 3.0 KB


In [5]:
chu_pre.describe()

,Unnamed: 0,연도,시간대,온도,풍속,시정,교통량,통행시간(분)
count,46.000000,46.000000,46.000000,46.000000,46.000000,46.000000,46.000000,46.000000
mean,22.500000,2020.521739,11.021739,21.634783,2.293478,1615.152174,175846.000000,281.913043
std,13.422618,0.505047,6.744509,3.568954,1.012456,545.462534,102185.802489,24.278775
min,0.000000,2020.000000,0.000000,16.200000,0.400000,92.000000,24182.000000,228.000000
25%,11.250000,2020.000000,5.250000,18.425000,1.725000,1221.000000,75400.250000,264.750000
50%,22.500000,2021.000000,11.000000,20.800000,2.150000,2000.000000,171936.500000,286.000000
75%,33.750000,2021.000000,16.750000,24.650000,2.700000,2000.000000,268434.750000,297.000000
max,45.000000,2021.000000,23.000000,28.000000,6.100000,2000.000000,337419.000000,326.000000


In [6]:
chu_pre.isnull().sum()

Unnamed: 0    0
연도            0
시간대           0
온도            0
풍속            0
시정            0
교통량           0
통행시간(분)       0
dtype: int64

In [7]:
chu_pre.corr()

,Unnamed: 0,연도,시간대,온도,풍속,시정,교통량,통행시간(분)
Unnamed: 0,1.000000,-0.865411,0.434852,-0.263440,-0.227049,-0.688236,0.229061,0.665402
연도,-0.865411,1.000000,0.074883,0.505045,0.419662,0.706421,0.029006,-0.530845
시간대,0.434852,0.074883,1.000000,0.383373,0.302348,-0.100128,0.507997,0.370227
온도,-0.263440,0.505045,0.383373,1.000000,0.701525,0.632597,0.797350,0.379263
풍속,-0.227049,0.419662,0.302348,0.701525,1.000000,0.358020,0.523997,0.261423
시정,-0.688236,0.706421,-0.100128,0.632597,0.358020,1.000000,0.134356,-0.292314
교통량,0.229061,0.029006,0.507997,0.797350,0.523997,0.134356,1.000000,0.767410
통행시간(분),0.665402,-0.530845,0.370227,0.379263,0.261423,-0.292314,0.767410,1.000000


In [6]:
chu_pre.drop('Unnamed: 0', axis = 1, inplace = True)
chu_pre.head()

,연도,시간대,온도,풍속,시정,교통량,통행시간(분)
0,2021,0,20.3,1.8,2000,53920,228
1,2021,1,20.1,2.0,2000,35778,240
2,2021,2,19.9,2.6,2000,26782,243
3,2021,3,19.5,2.1,2000,24182,242
4,2021,4,19.2,1.7,2000,30804,254


In [7]:
target = '통행시간(분)'

#x, y 분리 
x = chu_pre.drop('통행시간(분)', axis = 1)
y= chu_pre[target]

In [8]:
from sklearn.model_selection import train_test_split

#7:3 분리 
x_train, x_test, y_train , y_test = train_test_split(x, y, test_size= 0.3)

In [9]:
#검증
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import cross_val_score

#선언 
model = DecisionTreeRegressor()

#검증하기 
cv_score = cross_val_score(model, x_train , y_train , cv = 10)

#확인
print(cv_score)
print(abs(cv_score.mean()))

[-4.29549486  0.78248974  0.68023788 -2.4375      0.81828275  0.39027027
  0.0672016   0.79726694  0.93104233 -0.69124024]
0.29574435814213007


In [10]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import cross_val_score

model = KNeighborsRegressor()

cv_score = cross_val_score(model, x_train, y_train, cv = 10)

print(cv_score)
print(abs(cv_score.mean()))

[-1.0030933   0.79964432  0.84470265 -0.74817308  0.94803849 -1.23063784
  0.79177533  0.74582553  0.86738279  0.65113617]
0.2666601076573163


In [11]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score

model = LinearRegression()

cv_score = cross_val_score(model, x_train, y_train ,cv = 10)

print(cv_score)
print(abs(cv_score.mean()))

[ 0.78364694  0.93849843  0.24315184 -0.43978806  0.94744991  0.81243539
  0.77898898  0.9804234   0.8270905   0.65964801]
0.6531545334780388


In [12]:
from sklearn.metrics import r2_score, mean_absolute_error

model = LinearRegression()

model.fit(x_train, y_train)

y_pred = model.predict(x_test)

print(y_pred)
print(r2_score(y_test, y_pred))
print(mean_absolute_error(y_test, y_pred))

[287.07331495 293.13525472 271.12709961 248.74046307 279.39494275
 273.49368169 299.8333204  310.57567118 273.08541832 285.19306073
 245.16161505 284.37850062 294.31787184 304.63483152]
0.8367902339287121
7.734587885820864
